<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/ger%C3%A7ek_zamanl%C4%B1_y%C3%BCz_tan%C4%B1ma_ve_duygu_analizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Adım 1: Gerekli kütüphaneleri yükleme
!pip install opencv-python numpy matplotlib deepface

# Adım 2: Güncellenmiş Yüz ve Duygu Analizi Sistemi
import cv2
import numpy as np
from deepface import DeepFace
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import time

# Hata yönetimi için geliştirilmiş analiz fonksiyonu
def analyze_face(img):
    try:
        # Sadece duygu analizi yapıyoruz (yaş ve cinsiyet çıkardık)
        analysis = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)

        if isinstance(analysis, list):
            analysis = analysis[0]

        return {
            'emotion': analysis['dominant_emotion'],
            'confidence': analysis['emotion'][analysis['dominant_emotion']]
        }
    except Exception as e:
        print(f"Analiz hatası: {str(e)}")
        return None

# Optimize edilmiş kamera yakalama
def get_webcam_image():
    js = Javascript('''
    async function capture() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Sadece 1 saniye bekleyerek daha hızlı yanıt
      await new Promise((resolve) => {
        setTimeout(resolve, 1000);
      });

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', 0.8);
    }
    ''')
    display(js)
    data = eval_js('capture()')
    return data

# Geliştirilmiş AI Tepki Sistemi
def emotion_analysis():
    print("⏳ Sistem başlatılıyor... Lütfen kameraya bakın")

    try:
        start_time = time.time()
        image_data = get_webcam_image()

        if not image_data:
            print("❌ Görüntü alınamadı")
            return

        # Görüntüyü işle
        binary_data = b64decode(image_data.split(',')[1])
        img_array = np.frombuffer(binary_data, dtype=np.uint8)
        img = cv2.imdecode(img_array, flags=cv2.IMREAD_COLOR)

        # Analiz yap
        result = analyze_face(img)

        if not result:
            print("⚠️ Yüz analizi başarısız oldu. Lütfen tekrar deneyin")
            return

        emotion = result['emotion']
        confidence = result['confidence']

        # Duyguya özel tepkiler
        responses = {
            'happy': ["Harika görünüyorsun! 😊", "Mutluluğun yüzünden okunuyor! 🌟"],
            'sad': ["Üzüntülü görünüyorsun... 😔", "Senin için bir şeyler yapabilir miyim?"],
            'angry': ["Sakin ol... Derin bir nefes al. 🧘", "Öfke geçicidir, sakin kalmaya çalış."],
            'surprise': ["Vay canına! Şaşırmış görünüyorsun. 😲", "Beklenmedik bir şey mi oldu?"],
            'fear': ["Korkma, güvendesin. 🤗", "Endişelenme, her şey yoluna girecek."],
            'neutral': ["Duygularını tam okuyamadım. 😐", "Nötr bir ifaden var."]
        }

        # Rastgele bir yanıt seç
        import random
        response = random.choice(responses.get(emotion, ["İlginç bir ifade!"]))

        # Sonuçları göster
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10,6))
        plt.imshow(img_rgb)
        plt.title(f"{response}\nDuygu: {emotion} (%{confidence:.1f} doğruluk)")
        plt.axis('off')
        plt.show()

        print(f"⏱️ Analiz süresi: {time.time()-start_time:.2f} saniye")

    except Exception as e:
        print(f"⚠️ Beklenmedik hata: {str(e)}")

# Sistemi çalıştır
emotion_analysis()